In [6]:
### Working mapper/inserter for MyTCGA db ########################################################################

################################################################################################################
# Question: Maybe smarter to start a new collection for MAF data and map it over the case ids? #################  
################################################################################################################

# Current ouput
# {
#   _id: ObjectId("642c349933e932b48ad5d1c3"),
#   Tumor_Sample_UUID: '8577ac01-1274-4bd5-ab04-380eaa78d95b',
#   Matched_Norm_Sample_UUID: '867338db-8269-4f32-b4f2-1c4b9d2566f2',
#   case_id: 'becedbfd-b2aa-4dde-b7f4-29e6f59ec32c',
#     info:[{
#         NCBI_Build: 'GRCh38',
#         Chromosome: 'chr1',
#         Start_Position: 31919026,
#         End_Position: 31919026,
#         Strand: '+',
#         Variant_Classification: 'Missense_Mutation',
#         Variant_Type: 'SNP',
#         Reference_Allele: 'T',
#         Tumor_Seq_Allele1: 'T',
#         Tumor_Seq_Allele2: 'A',
#         HGVSc: 'c.40A>T',
#         HGVSp_Short: 'p.M14L'
#         }]
# }
        
def inserter():
    bs.insert_one({
        'Tumor_Sample_UUID': sample_id,
        'Matched_Norm_Sample_UUID': mns_id,
        'case_id': case_id,
        'info': [{
            'NCBI_Build': build,
            'Chromosome': chro,
            'Start_Position': start,
            'End_Position': end,
            'Strand': strand,
            'Variant_Classification': vc,
            'Variant_Type': vt,
            'Reference_Allele': refall,
            'Tumor_Seq_Allele1': tall1,
            'Tumor_Seq_Allele2': tall2,
            'HGVSc': hgvsc,
            'HGVSp': hgvsp
        }]
    })
        
def wanna_see(matches):
    print("Do you want to see the matches? [Y/N]")
    x = input()
    match = matches
    if x in ['Y', 'y', 'yes', 'Yes']:
        for obj in matches:
            print("\n")
            pprint.pprint(obj)
    if x in ['N', 'n', 'no', 'No']:
        print("Proceeding...")
    print("\n")
    

import pandas as pd
import pprint
from pymongo import MongoClient

# Prepare mongodb tools
client = MongoClient('localhost', 27017)
db = client.MyTCGA 
bs = db[ "biosamples" ]

# Read in dataframe from mapping file
df = pd.read_csv('../temp/intermediate_mapping_file.csv') ###### Make input later

# iterate through the df and extract information
for row, line in df.iterrows():
    sample_id = line[0]
    mns_id = line[1]
    case_id = line[2]
    build = line[3]
    chro = line[4]
    start = line[5]
    end = line[6]
    strand = line[7]
    vc = line[8]
    vt = line[9]
    refall = line[10]
    tall1 = line[11]
    tall2 = line[12]
    hgvsc = line[13]
    hgvsp = line[14]
    info = line[3:15]
    
    case_hit = bs.find_one({"case_id": case_id})
    case_hits = bs.find({"case_id": case_id})
    n_hits_c = bs.count_documents({"case_id": case_id})
    sample_hit = bs.find_one({"Tumor_Sample_UUID": sample_id})
    sample_hits = bs.find({"Tumor_Sample_UUID": sample_id})
    n_hits_s = bs.count_documents({"Tumor_Sample_UUID": sample_id})
    
    print("Checking database...\n")
    if case_hit is None:
        print("No matching case id found for: ", case_id)
        print("Inserting data to database\n")
        inserter()
    else:
        print("Case id:", case_id, "in database")
        print("Number of matches with case id =", n_hits_c, "\n")
        #wanna_see(case_hits)
    
        if sample_hit is None:
            print("No matching sample id found for:", sample_id)
            print("Inserting data to database\n")
            inserter()
        else:
            print("Sample id:", sample_id, "already in database")
            print("Number of matches with sample id =", n_hits_s, "\n")
            # wanna_see(sample_hits)
            
            if info.to_dict() in case_hit['info']:
                print('Data already logged\n')
                #wanna_see(case_hits)
            else:
                print('Inserting variant information to database\n')
                bs.update_many({'case_id': case_id}, 
                         {'$push': {
                            'info': {
                                'NCBI_Build': build,
                                'Chromosome': chro,
                                'Start_Position': start,
                                'End_Position': end,
                                'Strand': strand,
                                'Variant_Classification': vc,
                                'Variant_Type': vt,
                                'Reference_Allele': refall,
                                'Tumor_Seq_Allele1': tall1,
                                'Tumor_Seq_Allele2': tall2,
                                'HGVSc': hgvsc,
                                'HGVSp': hgvsp
                            }
                         }
                     })
                
    print("#" * 79 + "\n")

Checking database...

Case id: becedbfd-b2aa-4dde-b7f4-29e6f59ec32c in database
Number of matches with case id = 1 

Sample id: 8577ac01-1274-4bd5-ab04-380eaa78d95b already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: becedbfd-b2aa-4dde-b7f4-29e6f59ec32c in database
Number of matches with case id = 1 

Sample id: 8577ac01-1274-4bd5-ab04-380eaa78d95b already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: becedbfd-b2aa-4dde-b7f4-29e6f59ec32c in database
Number of matches with case id = 1 

Sample id: 8577ac01-1274-4bd5-ab04-380eaa78d95b already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...



Checking database...

Case id: b956df1b-bc77-4bdb-a5bd-5dda7b4c8da7 in database
Number of matches with case id = 1 

Sample id: e46a5d19-2dd7-4c34-8fff-6276278c58b3 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: b956df1b-bc77-4bdb-a5bd-5dda7b4c8da7 in database
Number of matches with case id = 1 

Sample id: e46a5d19-2dd7-4c34-8fff-6276278c58b3 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: b956df1b-bc77-4bdb-a5bd-5dda7b4c8da7 in database
Number of matches with case id = 1 

Sample id: e46a5d19-2dd7-4c34-8fff-6276278c58b3 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...



Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Inserting variant information to database

###############################################################################



Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...



Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...



Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c364e81c-eb1e-4870-ab37-9c661f5f2e3d in database
Number of matches with case id = 1 

Sample id: 63d635fa-d136-4e8a-a534-966ee678bb66 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...



Checking database...

Case id: c0892598-1f7b-4f23-9cd8-731f797753d5 in database
Number of matches with case id = 1 

Sample id: bf82035c-9cd1-4355-acdd-8a007708e976 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c0892598-1f7b-4f23-9cd8-731f797753d5 in database
Number of matches with case id = 1 

Sample id: bf82035c-9cd1-4355-acdd-8a007708e976 already in database
Number of matches with sample id = 1 

Data already logged

###############################################################################

Checking database...

Case id: c0892598-1f7b-4f23-9cd8-731f797753d5 in database
Number of matches with case id = 1 

Sample id: bf82035c-9cd1-4355-acdd-8a007708e976 already in database
Number of matches with sample id = 1 

Inserting variant information to database

###############################################################################



In [15]:
### Progenetix inserter/mapper ##########################################################
######## PROBLEM: same case_id & sample_id but still different info
# Current ouput
# {
#   _id: ObjectId("642c349933e932b48ad5d1c3"),
#   Tumor_Sample_UUID: '8577ac01-1274-4bd5-ab04-380eaa78d95b',
#   Matched_Norm_Sample_UUID: '867338db-8269-4f32-b4f2-1c4b9d2566f2',
#   case_id: 'becedbfd-b2aa-4dde-b7f4-29e6f59ec32c',
#   NCBI_Build: 'GRCh38',
#   Chromosome: 'chr1',
#   Start_Position: 31919026,
#   End_Position: 31919026,
#   Strand: '+',
#   Variant_Classification: 'Missense_Mutation',
#   Variant_Type: 'SNP',
#   Reference_Allele: 'T',
#   Tumor_Seq_Allele1: 'T',
#   Tumor_Seq_Allele2: 'A',
#   HGVSc: 'c.40A>T',
#   HGVSp_Short: 'p.M14L'
# }




def maf_mapper(input_file):
    
    def inserter():
        maf.insert_one({
            'Tumor_Sample_UUID': sample_id,
            'Matched_Norm_Sample_UUID': mns_id,
            'case_id': case_id,
            'info': [{
                'NCBI_Build': build,
                'Chromosome': chro,
                'Start_Position': start,
                'End_Position': end,
                'Strand': strand,
                'Variant_Classification': vc,
                'Variant_Type': vt,
                'Reference_Allele': refall,
                'Tumor_Seq_Allele1': tall1,
                'Tumor_Seq_Allele2': tall2,
                'HGVSc': hgvsc,
                'HGVSp': hgvsp
            }]
        })

    
    # Prepare modules
    import pandas as pd
    import pprint
    from pymongo import MongoClient

    # Prepare mongodb tools
    client = MongoClient('localhost', 27017)
    db = client.progenetixCopy ############################### Copy database
    maf = db[ "MAF" ] ######################################## MAF collection?

    # Read in dataframe from mapping file
    df = pd.read_csv(input_file) ###### Make input later

    for row, line in df.iterrows():
        sample_id = line[0]
        mns_id = line[1]
        case_id = line[2]
        build = line[3]
        chro = line[4]
        start = line[5]
        end = line[6]
        strand = line[7]
        vc = line[8]
        vt = line[9]
        refall = line[10]
        tall1 = line[11]
        tall2 = line[12]
        hgvsc = line[13]
        hgvsp = line[14]

        case_hit = maf.find_one({"case_id": case_id})
        case_hits = maf.find({"case_id": case_id})
        n_hits_c = maf.count_documents({"case_id": case_id})
        sample_hit = maf.find_one({"Tumor_Sample_UUID": sample_id})
        sample_hits = maf.find({"Tumor_Sample_UUID": sample_id})
        n_hits_s = maf.count_documents({"Tumor_Sample_UUID": sample_id})

        if case_hit is None:
            print("No matching case id found for: ", case_id)
            print("Inserting data to data bank\n")
            inserter()

        else:
            print("Case id:", case_id, "in database")
            print("Number of matches with case id =", n_hits_c)
            print("\n")

            if sample_hit is None:
                print("No matching sample id found for:", sample_id)
                print("Inserting data to data bank\n")
                inserter()

            else:
                print("Sample id:", sample_id, "in database")
                print("Number of matches with sample id =", n_hits_s, "\n")

        print("#" * 79 + "\n")

maf_mapper('../temp/intermediate_mapping_file.csv')

IndentationError: expected an indented block after function definition on line 28 (3809820694.py, line 29)

In [ ]:
# Database Progenetix
# - analysis # empty

# - beaconinfo
#     date; 
#     datasets: 
#         cellosaurus, 
#         dipg(
#             counts(biosamples,indi,var,var_dist), 
#             filtering_terms(source, id, label, count)), 
#         arraymaps(counts, filtering_terms), 
#         progenetix(counts, filtering_terms)

# - biosample # empty

# - biosamples 
#     id; 
#     description; 
#     updated; 
#     individual_id; 
#     data_use_condition; 
#     info: 
#         callset_ids, 
#         legacy_ids;
#     external_references; 
#     provenance(geolocation); 
#     histological_diagnosis; 
#     sampled_tissue; 
#     cohorts;
#     pathological_tnm_findings; 
#     tumor_grade; 
#     pathological_stage; 
#     icdo_morphology; 
#     icdo_topography;
#     followup_state; 
#     biosample_status; 
#     followup_time; 
#     time_of_collection

# - callsets 
#     id; 
#     description; 
#     biosample_id; 
#     data_use_conditions; 
#     updated; 
#     info(legacy_id); 
#     provenance(geolocation);
#     individual_id; 
#     platform_model; 
#     cnv_chro_status: 
#         chr1-Y(cnvcoverage, 
#                 dupcoverage, 
#                 delcoverage, 
#                 cnvfraction, 
#                 dupfraction, 
#                 delfraction)

# - collations ("NCIT") 
#     id; 
#     label; 
#     type; 
#     collation_type; 
#     namespace_prefix; 
#     scope; 
#     db_key; 
#     hierarchy_paths(order, depth, path)
#     parent_terms; 
#     child_terms; 
#     reference; 
#     code_matches; 
#     code; 
#     count; 
#     dataset_id; 
#     updated; 
#     cnv_analysis

# - frequencymaps
#     id; 
#     label; 
#     dataset_id; 
#     scope(collection); 
#     db_key; 
#     collation_type; 
#     child_terms; 
#     updated; 
#     counts; 
#     frequencymap: 
#         interval_count, 
#         binning, 
#         biosample_count, 
#         intervals: (1-100) 
#             no, 
#             id, 
#             reference, 
#             arm, 
#             cytobands, 
#             start, 
#             end, 
#             size,
#             gain_freq, 
#             gain_mean, 
#             gain_median, same for loss
#         cnv_statusmaps: 
#             interval_count, 
#             binning, 
#             dup, 
#             del, 
#             max, 
#             min

# - genes 
#     symbol; 
#     accession_version; 
#     annotations: 
#         assemblies_in_scope, 
#         release_date, 
#         release_name; 
#     end; 
#     ensembl_gene_ids; 
#     gene_id; 
#     gene_locus_length; 
#     genomic_ranges: 
#         accession_version, 
#         range(
#             begin, 
#             end, 
#             order, 
#             orientation, 
#             ribosomal_slippage);
#     nomenclature_authority: 
#         authority, 
#         identifier

# - genespans 
#     gene_entrez_id; 
#     gene_symbol; 
#     reference_name; 
#     end; 
#     start

# - geocountries
#     type; 
#     properites; 
#     geometry: 
#         type, 
#         coordinates

# - geolocs 
#     id; 
#     geo_location: 
#         type, 
#         geometry(
#             type, 
#             coordinates), 
#         properties(
#             a2, 
#             a3, 
#             city, 
#             continent, 
#             country)

# - individual # empty

# - individuals
#     id; 
#     description; 
#     data_use_conditions;
#     updated; 
#     info; 
#     provenance(geoloc); 
#     sex; 
#     external_ref;
#     vital_status; 
#     index_disease: 
#         disease_code, 
#         stage, 
#         clinical_tnm_finding, 
#         followup_status, 
#         followup_time, 
#         onset(age)

# - ontologymaps
#     id; 
#     examples(terms); 
#     code_group

# - publications
#     updated; 
#     provenance(geoloc); 
#     counts(ccgh, 
#            acgh, 
#            wes, 
#            wgs, 
#            ngs, 
#            genomes, 
#            progenetix, 
#            arraymap);
#     id; 
#     abstract; 
#     authors; 
#     journal; 
#     note; 
#     status; 
#     title; 
#     pubmedid; 
#     info; 
#     label; 
#     contact(name, email, affiliation)
#     progenetix_curator; 
#     progenetix_use; 
#     pub_year

# - querybuffer 
#     id; 
#     original_queries; 
#     source_collection; 
#     source_db; 
#     source_key; 
#     target_collection; 
#     target_count;
#     target_key; 
#     target_values

# - variant # empty

# - variants
#     callset_id; 
#     info; 
#     updated; 
#     biosample_id; 
#     id;
#     variant_internal_id; 
#     variant_state; 
#     type;
#     location: 
#         sequence_id, 
#         type, 
#         interval(start, end);
#         relative_copy_class; 
#         individual_id

##################################################################################################################

# Database Progenetix
# - analysis # empty

# - beaconinfo # date; datasets: cellosaurus, 
#                                dipg(counts(biosamples,indi,var,var_dist), filtering_terms(source, id, label, count)), 
#                                arraymaps(counts, filtering_terms), 
#                                progenetix(counts, filtering_terms)

# - biosample # empty

# - biosamples # id; description; updated; individual_id; data_use_condition; 
#                info: callset_ids, legacy_ids;
#                external_references; provenance(geolocation); histological_diagnosis; sampled_tissue; cohorts;
#                pathological_tnm_findings; tumor_grade; pathological_stage; icdo_morphology; icdo_topography;
#                followup_state; biosample_status; followup_time; time_of_collection

# - callsets # id; description; biosample_id; data_use_conditions; updated; info(legacy_id); provenance(geolocation);
#              individual_id; platform_model; cnv_chro_status: chr1-Y(cnvcoverage, dupcoverage, delcoverage,
#                                                                     cnvfraction, dupfraction, delfraction)

# - collations # "NCIT": id; label; type; collation_type; namespace_prefix; scope; db_key; hierarchy_paths(order, depth, path)
#                parent_terms; child_terms; reference; code_matches; code; count; dataset_id; updated; cnv_analysis

# - frequencymaps # id; label; dataset_id; scope(collection); db_key; collation_type; child_terms; updated; counts; 
#                    frequencymap: interval_count, binning, biosample_count, 
#                                  intervals: (1-100, no, id, ref, arm, cytobands, start, end, size,
#                                                   gain_freq, gain_mean, gain_median, same for loss)
#                   cnv_statusmaps: interval_count, binning, dup, del, max, min

# - genes # symbol; accession_version; annotations: assemblies_in_scope, release_date, release_name; end; 
#           ensembl_gene_ids; gene_id; gene_locus_length; 
#           genomic_ranges: accession_version, range(begin, end, order, orientation, ribosomal_slippage);
#           nomenclature_authority: authority, identifier

# - genespans # gene_entrez_id; gene_symbol; reference:name; end; start

# - geocountries # type; properites; geometry: type, coordinates

# - geolocs # id; geo_location: type, geometry(type, coordinates), properties(a2, a3, city, continent, country)

# - individual # empty

# - individuals # id; description; data_use_conditions: updated; info; provenance(geoloc); sex; external_ref;
#                 vital_status; 
#                 index_disease: disease_code, stage, clinical_tnm_finding, followup_status, followup_time, onset(age)

# - ontologymaps # id; examples(terms); code_group

# - publications # updated; provenance(geoloc); counts(ccgh, acgh, wes, wgs, ngs, genomes, progenetix, arraymap);
#                  id; abstract; authors; journal; note; status; title; pubmedid; info; label; contact(name, email, affiliation)
#                  progenetix_curator; progenetix_use; pub_year

# - querybuffer # id; original_queries; source_collection; source_db; source_key; target_collection; target_count;
#                 target_key; target_values

# - variant # empty

# - variants # callset_id; info; updated; biosample_id; id; variant_internal_id; variant_state; type;
#              location: sequence_id, type, interval(start, end);
#              relative_copy_class; individual_id
##################################################################################################################